# Associazione ID giocatore fra diversi fornitori

Purtroppo i diversi fornitori di dati calcistici attivi sul mercato non si sono uniformati su un sistema di ID numerici comuni per i giocatori. Questo presenta un problema nel momento in cui si vogliono combinare dati di fornitori diversi, applicazione molto comune quando si lavora in un club. Andiamo quindi a vedere come possiamo incrociare i dati a disposizione per creare una mappatura fra gli ID numerici di fornitori diversi, utilizzando i dati SICS e Opta come esempio.

In [ ]:
import pandas as pd
import numpy as np
import requests
from requests.auth import HTTPBasicAuth
from fuzzywuzzy import fuzz
from unidecode import unidecode
from database.read_db import *

Leggiamo la tabella SICS con i dati anagrafici dei giocatori nel 2022, usando le API introdotte nella lezione precedente, andando a inserirle in un dataframe.

In [ ]:
season_to_search = 2022
player_link = f'http://server.sics.it/livefeeds/s02.json?seasonId={season_to_search}&lang=en'

In [ ]:
dfp = pd.DataFrame(requests.get(player_link, auth = HTTPBasicAuth('roberto.angioni@soccerment.com', 'yozBj(9RfHX_xQ0')).json()['player'])
dfp.head()

In questa tabella troviamo informazioni anagrafiche come nome e cognome dei giocatori, data di nascita e nazionalità, insieme all'identificativo numerico univoco, contenuto nella colonna `id`.

Ora andiamo a leggere l'anagrafica dei giocatori Opta, che abbiamo selezionato dai dati Soccerment prendendo tutti i giocatori con almeno un minuto nella Serie A 2022/23.

In [ ]:
dfpi = read_db('ds.player_registry')

In [ ]:
dfpi.head()

Vediamo che abbiamo a disposizione dati simili a quelli dell'anagrafica SICS, con l'idenfiticativo numerico di Opta sotto la colonna `player_id`. L'operazione che vogliamo svolgere è l'associazione dei due ID numerici in modo da poterli mappare uno sull'altro e quindi unire dataset provenienti dai due fornitori in modo corretto.

# Preparazione dei dati

Prima di impostare l'operazione di associazione degli ID, è importante verificare l'integrità del dato da entrambi i lati, e applicare accorgimenti se necessario. Prima di tutto è bene verificare se ci sono giocatori duplicati, cosa che può capitare e che in questo caso non vogliamo: ci serve una riga sola per giocatore. Iniziamo controllando i dati SICS:

In [ ]:
dfp[dfp.id.duplicated(keep = False)]

Vediamo che ci sono parecchi giocatori con righe duplicate, per cui andiamo a eliminare i doppioni con la funzione `drop_duplicates` di Pandas, tenendo una sola riga per ogni `id` giocatore univoco.

In [ ]:
dfp = dfp.drop_duplicates(subset = ['id'])

Facciamo lo stesso controllo anche sui dati Opta:

In [ ]:
dfpi[dfpi.player_id.duplicated()]

Vediamo che in questo caso non abbiamo doppioni. 

La seconda operazione che è bene fare è la conversione delle date di nascita da entrambi i lati in un formato comune. Questa è buona pratica anche se in questo caso il formato sembra simile (YYYY-MM-DD), visto che andremo a cercare una corrispondenza esatta fra le date. Usiamo la funzione `to_datetime` di Pandas per questa operazione:

In [ ]:
#convertiamo la data di nascita in un formato standardizzato di Pandas
dfp['bornDate'] = pd.to_datetime(dfp.bornDate)
dfpi['birth_date'] = pd.to_datetime(dfpi.birth_date)

Proviamo in prima battuta a fare un `merge` utilizzando la funzione apposita di Pandas sulle colonne di nome, cognome e data di nascita:

In [ ]:
dfpi2 = dfpi.merge(dfp, left_on = ['first_name', 'last_name', 'birth_date'], right_on = ['firstName', 'lastName', 'bornDate'], how = 'inner')
dfpi2.head()

Vediamo che non troviamo corrispondenza, perché le stringhe dei nomi sono tutte in maiuscolo lato SICS. Proviamo a trasformare le stringhe mettendo tutti i caratteri in minuscolo da entrambi i lati:

In [ ]:
for c in ['first_name', 'last_name']:
    dfpi[c] = dfpi[c].str.lower()

for c in ['firstName', 'lastName']:
    dfp[c] = dfp[c].str.lower()

dfpi2 = dfpi.merge(dfp, left_on = ['first_name', 'last_name', 'birth_date'], right_on = ['firstName', 'lastName', 'bornDate'], how = 'inner')
dfpi2.head()

Anche in questo caso troviamo solo 5 associazioni. Questo avviene perché le stringhe possono avere comunque differenze: abbiamo quindi bisogno di un sistema di associazione più flessibile rispetto a un `merge` vero e proprio, che vada a ricercare per ogni giocatore da un lato la corrispondenza migliore fra tutti i giocatori dall'altro lato. L'opzione più utilizzata in questi casi è quella di fare ricorso a delle librerie che valutano la similarità fra stringhe, ricavando un punteggio quantitativo. Una delle librerie più utilizzate in questo ambito è `fuzzywuzzy`. 

Andiamo a vedere come possiamo applicare questo strumento al nostro caso d'uso, sfruttando le informazioni su nome, cognome e nazionalità dei giocatori, oltre alla data di nascita. Usiamo l'anagrafica Opta come punto di partenza, andando a ricercare per ogni giocatore l'associazione migliore nell'anagrafica SICS. In questo processo, è buona pratica andare a normalizzare le stringhe, sostituendo eventuali caratteri speciali (accenti, lettere nordiche etc.): nel nostro esempio lo facciamo tramite la libreria `unidecode`.

Torniamo prima ai nostri dataframe originali andando a leggerli nuovamente:

In [ ]:
dfp = pd.DataFrame(requests.get(player_link, auth = HTTPBasicAuth('roberto.angioni@soccerment.com', 'yozBj(9RfHX_xQ0')).json()['player'])
dfp.drop_duplicates(subset = ['id'], inplace = True)

dfpi = read_db('ds.player_registry')

dfp['bornDate'] = pd.to_datetime(dfp.bornDate)
dfpi['birth_date'] = pd.to_datetime(dfpi.birth_date)

In [ ]:
# Ciclo su tutti i giocatori della lista di giocatori di Opta
for index, row in dfpi.iterrows():
    print(f'Processing player {index+1}/{len(dfpi)}...', end = '\r')
    
    # Estrazione informazioni giocatore
    first_name = unidecode(row['first_name'])
    last_name = unidecode(row['last_name'])
    country = unidecode(row['country'])
    birth_date = row['birth_date']
    
    # Calcolo delle similarità tra nomi e nazionalità + corrispondenza data di nascita
    dfp2 = dfp.copy()
    dfp2['match_firstName'] = dfp2.apply(lambda x: fuzz.token_set_ratio(unidecode(x['firstName']), first_name), axis = 1)
    dfp2['match_lastName']  = dfp2.apply(lambda x: fuzz.token_set_ratio(unidecode(x['lastName']), last_name), axis = 1)
    dfp2['match_citizenship'] = dfp2.apply(lambda x: fuzz.token_set_ratio(unidecode(x['citizenship']), country), axis = 1)
    dfp2['match_bornDate'] = dfp2.bornDate == birth_date

    # Calcolo della similarità media
    dfp2['similarity'] = dfp2[['match_firstName', 'match_lastName', 'match_citizenship']].mean(axis = 1)

    # Filtro per età
    dfp3 = dfp2[dfp2.match_bornDate].sort_values('similarity', ascending = False)

    # Se c'è almeno un match, seleziono il migliore
    if len(dfp3) > 0:
        best_match = dfp3.iloc[0]
        dfpi.at[index, 'sics_id'] = best_match['id']
        dfpi.at[index, 'sics_firstName'] = best_match['firstName']
        dfpi.at[index, 'sics_lastName'] = best_match['lastName']
        dfpi.at[index, 'sics_citizenship'] = best_match['citizenship']
        dfpi.at[index, 'similarity'] = best_match['similarity']
        # Se c'è ambiguità, stampo i risultati
        if len(dfp3) > 1 and best_match['similarity'] < 70.:
            print('Ambiguity for ', first_name, last_name, country, birth_date)
            print(dfp3[['firstName', 'lastName', 'citizenship', 'bornDate', 'similarity']])
            print('Selected: ', best_match[0]['firstName'], best_match[0]['lastName'], best_match[0]['citizenship'], best_match[0]['bornDate'])
            print('')
    else:
        # Se non c'è match, stampo i risultati
        print('No match for ', first_name, last_name, country, birth_date)
        print('')

Vediamo che questo metodo riesce a trovare un'associazione soddisfacente per tutti i giocatori tranne uno, senza casi ambigui. È probabile che su un dataset più ampio ci saranno più casi ambigui, che andranno inseriti a mano, ma dovrebbero comunque essere una minoranza.

Vediamo se effettivamente il giocatore mancante non è presente nei dati SICS:

In [ ]:
dfp[dfp.lastName == 'KARAMOH']

In questo caso vediamo che l'associazione è fallita perché la data di nascita del giocatore è sbagliata, un caso che può sempre capitare. Possiamo comunque gestire questo caso inserendo un ulteriore `if` nel nostro codice, per cui se non viene trovata un'associazione andiamo a riprovare escludendo il filtro sulla data di nascita, usando quindi solo il punteggio di similarità:

In [ ]:
dfp = pd.DataFrame(requests.get(player_link, auth = HTTPBasicAuth('roberto.angioni@soccerment.com', 'yozBj(9RfHX_xQ0')).json()['player'])
dfp.drop_duplicates(subset = ['id'], inplace = True)

dfpi = read_db('ds.player_registry')

dfp['bornDate'] = pd.to_datetime(dfp.bornDate)
dfpi['birth_date'] = pd.to_datetime(dfpi.birth_date)

In [ ]:
# Ciclo su tutti i giocatori della lista di giocatori di Opta
for index, row in dfpi.iterrows():
    print(f'Processing player {index+1}/{len(dfpi)}...', end = '\r')
    
    # Estrazione informazioni giocatore
    first_name = unidecode(row['first_name'])
    last_name = unidecode(row['last_name'])
    country = unidecode(row['country'])
    birth_date = row['birth_date']
    
    # Calcolo delle similarità tra nomi e nazionalità + corrispondenza data di nascita
    dfp2 = dfp.copy()
    dfp2['match_firstName'] = dfp2.apply(lambda x: fuzz.token_set_ratio(unidecode(x['firstName']), first_name), axis = 1)
    dfp2['match_lastName']  = dfp2.apply(lambda x: fuzz.token_set_ratio(unidecode(x['lastName']), last_name), axis = 1)
    dfp2['match_citizenship'] = dfp2.apply(lambda x: fuzz.token_set_ratio(unidecode(x['citizenship']), country), axis = 1)
    dfp2['match_bornDate'] = dfp2.bornDate == birth_date

    # Calcolo della similarità media
    dfp2['similarity'] = dfp2[['match_firstName', 'match_lastName', 'match_citizenship']].mean(axis = 1)

    # Filtro per età
    dfp3 = dfp2[dfp2.match_bornDate].sort_values('similarity', ascending = False)

    # Se c'è almeno un match, seleziono il migliore
    if len(dfp3) > 0:
        best_match = dfp3.iloc[0]
        dfpi.at[index, 'sics_id'] = best_match['id']
        dfpi.at[index, 'sics_firstName'] = best_match['firstName']
        dfpi.at[index, 'sics_lastName'] = best_match['lastName']
        dfpi.at[index, 'sics_citizenship'] = best_match['citizenship']
        dfpi.at[index, 'similarity'] = best_match['similarity']
        # Se c'è ambiguità, stampo i risultati
        if len(dfp3) > 1 and best_match['similarity'] < 70.:
            print('Ambiguity for ', first_name, last_name, country, birth_date)
            print(dfp3[['firstName', 'lastName', 'citizenship', 'bornDate', 'similarity']])
            print('Selected: ', best_match[0]['firstName'], best_match[0]['lastName'], best_match[0]['citizenship'], best_match[0]['bornDate'])
            print('')
    else:
        # Se non c'è match, provo a rimuovere il filtro sulla data di nascita
        dfp3 = dfp2.sort_values('similarity', ascending = False)
        if len(dfp3) > 0:
            best_match = dfp3.iloc[0]
            dfpi.at[index, 'sics_id'] = best_match['id']
            dfpi.at[index, 'sics_firstName'] = best_match['firstName']
            dfpi.at[index, 'sics_lastName'] = best_match['lastName']
            dfpi.at[index, 'sics_citizenship'] = best_match['citizenship']
            dfpi.at[index, 'similarity'] = best_match['similarity']
            # Se c'è ambiguità, stampo i risultati
            if len(dfp3) > 1 and best_match['similarity'] < 70.:
                print('Ambiguity for ', first_name, last_name, country, birth_date)
                print(dfp3[['firstName', 'lastName', 'citizenship', 'bornDate', 'similarity']])
                print('Selected: ', best_match[0]['firstName'], best_match[0]['lastName'], best_match[0]['citizenship'], best_match[0]['bornDate'])
                print('')
        else:
            # Se non c'è match, stampo i risultati
            print('No match for ', first_name, last_name, country, birth_date)
            print('')

Vediamo che il caso del giocatore mancante viene ora gestito correttamente, e non ci sono altri buchi:

In [ ]:
dfpi[dfpi.last_name == 'Karamoh']

È comunque buona pratica fare un controllo manuale sulle associazioni, soprattutto quelle dove il punteggio di similarità non è altissimo. Nel nostro caso andiamo a vedere tutti i casi con similarità sotto il 100%:

In [ ]:
dfpi[dfpi.similarity < 100.].sort_values('similarity')

Vediamo che in molti casi la discrepanza emerge sulla nazionalità: anche qui è normale che fornitori diversi usino formulazioni diverse del nome dei paesi, per cui sarebbe ideale avere dei codici paese standardizzati per questa operazione. Ci sono anche casi di nazionalità discordanti, dovute a eventuali cambi di passaporto o nazionale di riferimento del giocatore, o interpretazioni diverse del concetto di nazionalità, fra cittadinanza alla nascita, nazionale in cui si gioca etc.